In [13]:
import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings("ignore")

In [14]:
names_df = pd.read_excel('./gu_list/서울_송파구.xlsx')
names_df

,구,동,사업장명,업종,좌표정보(X),좌표정보(Y),리뷰주소
0,송파구,신천동,아바이 순대국,한식,208707.089898,446299.352775,https://place.map.kakao.com/899797623#comment
1,송파구,잠실동,띠아 데 부리또,분식,207373.500215,445545.656024,https://place.map.kakao.com/1173204825#comment
2,송파구,잠실동,브이카페,한식,207361.464510,445173.152754,https://place.map.kakao.com/1351088138#comment
3,송파구,삼전동,청년어부 삼전점,일식,207761.510501,444639.252711,https://place.map.kakao.com/631901539#comment
4,송파구,잠실동,포커플레이스,치킨,207052.375771,445360.804502,https://place.map.kakao.com/1474039836#comment
...,...,...,...,...,...,...,...
5441,송파구,삼전동,스테이크&샐러드전문점 파파그릴,카페,207765.345117,444230.811991,https://place.map.kakao.com/26638886#comment
5442,송파구,잠실동,신천1호점(하남돼지집),한식,207170.295764,445305.329985,https://place.map.kakao.com/27029368#comment
5443,송파구,가락동,상수씨(SANS SOUCI),카페,211799.818800,443487.198513,https://place.map.kakao.com/2062269131#comment
5444,송파구,방이동,이가장,한식,210432.104382,445272.701865,https://place.map.kakao.com/1416116010#comment


In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('headless')

path = 'C:/Users/USER/TIL/chromedriver'

source_url = "https://map.kakao.com/"

page_urls = []

driver = webdriver.Chrome(executable_path=path, chrome_options=options)

for name in tqdm_notebook(names_df['0']):
    try:
        driver.get(source_url)

        search_bar = driver.find_element_by_xpath("//input[@id='search.keyword.query']")
        search_bar.send_keys(name)

        searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']")
        driver.execute_script("arguments[0].click();", searchbutton)

        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="info.search.place.list"]/li[2]/div[3]/strong/a[1]')))

        html = driver.page_source

        soup = BeautifulSoup(html, "html.parser")
        moreview = soup.find(name="a", attrs={"class":"moreview"})
    
        page_url = moreview.get("href")
        print(page_url)
        page_urls.append(page_url)
    except:
        pass

driver.close()

https://place.map.kakao.com/207990934
https://place.map.kakao.com/1441083691
https://place.map.kakao.com/1605389572
https://place.map.kakao.com/908713570
https://place.map.kakao.com/16738793
https://place.map.kakao.com/21413534
https://place.map.kakao.com/1824464463
https://place.map.kakao.com/15558781
https://place.map.kakao.com/21753236
https://place.map.kakao.com/8607176
https://place.map.kakao.com/274655501
https://place.map.kakao.com/7902292
https://place.map.kakao.com/16296316


In [15]:
page_urls = []

for i in names_df.리뷰주소:
    page_urls.append(i)

options = webdriver.ChromeOptions()
options.add_argument('headless')

path = 'C:/Users/USER/TIL/chromedriver'

In [16]:
columns = ['name', 'score', 'review']
review_df = pd.DataFrame(columns=columns)

driver = webdriver.Chrome(executable_path=path, chrome_options=options)

for page_url in tqdm_notebook(page_urls):
    try:
        driver.get(page_url)
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="mArticle"]/div[1]/div[1]/div[2]/div/h2')))
        

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        names = soup.find(name='h2', attrs={"class":"tit_location"})
        
        contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})
        rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})
        reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})
    
    
        for rate, review in zip(rates, reviews):
            if review != ' ':
                row = [names.text, rate.text[0], review.find(name="span").text]
                series = pd.Series(row, index=review_df.columns)
                review_df = review_df.append(series, ignore_index=True)

        for button_num in range(2, 6):
            another_reviews = driver.find_element_by_xpath("//a[@data-page='" + str(button_num) + "']")
            another_reviews.click()
            driver.implicitly_wait(5)
            
        
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')

            contents_div = soup.find(name="div", attrs={"class":"evaluation_review"})
            rates = contents_div.find_all(name="em", attrs={"class":"num_rate"})
            reviews = contents_div.find_all(name="p", attrs={"class":"txt_comment"})

            for rate, review in zip(rates, reviews):
                if review != ' ':
                    row = [names.text, rate.text[0], review.find(name="span").text]
                    series = pd.Series(row, index=review_df.columns)
                    review_df = review_df.append(series, ignore_index=True)
    except:
        pass
    
driver.close()

  0%|          | 0/5446 [00:00<?, ?it/s]

In [18]:
review_df['review'].replace(' ', np.nan, inplace=True)
review_data = review_df.dropna(subset=['review'])
review_data

,name,score,review
0,아바이순대국,5,순대국 국물이 진하고 내용물이 많아서 맛있어요 매장이 넓어서 식사하는데 마음이 편합...
1,아바이순대국,3,
2,아바이순대국,3,친절
3,아바이순대국,5,"순대국은 국물도 진하고 고기도 맛있고, 순대도 맛있고 칡냉면은 육수와 다대기가 다 ..."
4,띠아 데 부리또,5,양도 푸짐하고 부리또도 쫄깃한게 너무 맛있게 먹었습니다. 하나먹고 배터지는줄 주말 ...
...,...,...,...
15930,하남돼지집 신천1호점,5,직접 구워줘서 너무 편해요. 고기굽기 장인들~~~ 서브메뉴들도 하나같이 맛있어요!
15931,하남돼지집 신천1호점,4,고기맛났음~고기구워주시구 굳 12명이서 회식하기 적당
15932,하남돼지집 신천1호점,3,걍 하남돼지임.. 가격대비 평타
15933,하남돼지집 신천1호점,1,이렇게까지 싸가지없고 불친절한곳은 처음이네요


In [6]:
review_data.to_csv("review_data.csv", index=False)

In [17]:
review_data.to_csv("./gu_list/송파구_review_data.csv", index=False, encoding='utf-8-sig')